# Challenge 04-A - Retrieval Augmented Generation (RAG) for Structured Data


## Introduction

In this notebook, we will explore the practical application of RAG with a more manageable type of data i.e structured data such as relational data or text data stored in csv files. The main objective is to introduce a specific use case that demonstrates the utilization of Azure Cognitive Search to extract relevant documents and the power of ChatGPT to address relevant portions of the document, providing concise summaries based on user prompts. It aims to showcase how Azure OpenAI's ChatGPT capabilities can be adapted to suit your summarization needs, while also guiding you through the setup and evaluation of summarization results. This method can be customized to suit various summarization use cases and applied to diverse datasets.

Your goals for this challenge are to read through this notebook, run each code block, observe the results, and then be able to answer the questions posed in the student guide.

## Use Case

This use case consists of three sections:
- Document Search - The process of extracting relevant documents based on the query from a corpus of documents.
- Document Zone Search - The process of finding the relevant part of the document extracted from document search.
- Downstream AI tasks such as Question Answering (aka Text summarization) - Text summarization is the process of creating summaries from large volumes of data while maintaining significant informational elements and content value.
This use case can be useful in helping subject matter experts in finding relevant information from large document corpus.

**Example:** In the drug discovery process, scientists in pharmaceutical industry read a corpus of documents to find specific information related to concepts, experiment results etc. This use case enables them to ask questions from the document corpus and the solution will come back with the succinct answer. Consequently, expediting the drug discovery process.
 
Benefits of the solution:
1. Shortens reading time
2. Improves the effectiveness of searching for information
3. Removes bias from human summarization techniques
4. Increases bandwidth for humans to focus on more in-depth analysis 


The need for document summarization be applied to any subject matter (legal, financial, journalist, medical, academic, etc) that requires long document summarization. The subject matter that this notebook is focusing on is journalistic - we will walk through news articles.   


### CNN & Daily Mail Dataset
For this walkthrough, we will be using the CNN/Daily Mail dataset. This is a common dataset used for text summarization and question answering tasks. Human generated abstractive summary bullets were generated from news stories on the CNN and Daily Mail websites.


### Data Description
The relevant schema for our work today consists of:

- `id`: a string containing the heximal formatted SHA1 hash of the URL where the story was retrieved from
- `article`: a string containing the body of the news article
- `highlights`: a string containing the highlight of the article as written by the article author


In [ ]:
# Import Azure Cognitive Search, OpenAI, and other python modules

import os, json, requests, sys, re
import requests
from pprint import pprint
import pandas as pd
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    SemanticConfiguration,
    PrioritizedFields,
    SemanticField,
    SemanticSettings
)


import openai
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity

from dotenv import load_dotenv
load_dotenv()

In [ ]:
# This is secure and recommended way to load OpenAI resource credentials and deployment names

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

chat_model = os.environ['CHAT_MODEL_NAME']
embedding_model=os.environ['EMBEDDING_MODEL_NAME']

**NOTE:** The path in the code cell below is referring to the `cnn_dailymail.csv` file in the `/data/structured/` folder. You may need to update this path if you are running this notebook from a different location.

In [ ]:
# read the CNN dailymail dataset in pandas dataframe
df = pd.read_csv('../data/structured/cnn_dailymail_data.csv') #path to CNN daily mail dataset
df.head()

In [ ]:
# Create a Cognitive Search Index client
service_endpoint = os.getenv("AZURE_COGNITIVE_SEARCH_ENDPOINT")   
key = os.getenv("AZURE_COGNITIVE_SEARCH_KEY")
credential = AzureKeyCredential(key)

index_name = "news-index"

index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)
index_client

### Define Index Fields and Create a Semantic Configuration

A *semantic configuration* specifies how fields are used in semantic ranking. It gives the underlying models hints about which index fields are most important for semantic ranking, captions, highlights, and answers.

You can add or update a semantic configuration at any time without rebuilding your index. When you issue a query, you'll add the semantic configuration (one per query) that specifies which semantic configuration to use for the query.

Review the properties you'll need to specify. A semantic configuration has a name and at least one each of the following properties:

* Title field - A title field should be a concise description of the document, ideally a string that is under 25 words. This field could be the title of the document, name of the product, or item in your search index. If you don't have a title in your search index, leave this field blank.
* Content fields - Content fields should contain text in natural language form. Common examples of content are the body of a document, the description of a product, or other free-form text.
* Keyword fields - Keyword fields should be a list of keywords, such as the tags on a document, or a descriptive term, such as the category of an item.

You can only specify one title field but you can specify as many content and keyword fields as you like. For content and keyword fields, list the fields in priority order because lower priority fields may get truncated.

In [ ]:
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="highlights", type=SearchFieldDataType.String,
                searchable=True, retrievable=True),
    SearchableField(name="article", type=SearchFieldDataType.String,
                filterable=True, searchable=True, retrievable=True),
]

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        #title_field=SemanticField(field_name=""), # title field is not present in the dataset. We can use OpenAI to generate title
        #prioritized_keywords_fields=[SemanticField(field_name="")], # keywords are not present in the dataset. We can use OpenAI to generate keywords
        prioritized_content_fields=[SemanticField(field_name="article"), SemanticField(field_name="highlights")]
    )
)


# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

In [ ]:
# Convert the dataframe to a list of dictionaries
documents = df.to_dict('records')
documents[0]

In [ ]:
len(documents)

In [ ]:
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents)  
print(f"Uploaded and Indexed {len(result)} documents") 

## Section 1: Leveraging Cognitive Search to extract relevant article based on the query 

In [ ]:
#Extracting relevant article based on the query. eg: Clinton Democratic Nomination
results = search_client.search(search_text="Clinton Democratic nomination", include_total_count=True)
document = next(results)['article']

In [ ]:
document

In [ ]:
#length of article extracted from Azure Cognitive search
len(document) 

## Section 2: Document Zone Search
### Document Zone: Azure OpenAI Embedding API
Now that we narrowed on a single document from our knowledge base using Azure Cognitive Search, we can dive deeper into the single document to refine our initial query to a more specific section or "zone" of the article.

To do this, we will utilize the Azure Open AI Embeddings API.

### **Embeddings Overview**
An embedding is a special format of data representation that can be easily utilized by machine learning models and algorithms. The embedding is an information dense representation of the semantic meaning of a piece of text. Each embedding is a vector of floating-point numbers, such that the distance between two embeddings in the vector space is correlated with semantic similarity between two inputs in the original format. For example, if two texts are similar, then their vector representations should also be similar.

Different Azure OpenAI embedding models are specifically created to be good at a particular task. Similarity embeddings are good at capturing semantic similarity between two or more pieces of text. Text search embeddings help measure long documents are relevant to a short query. Code search embeddings are useful for embedding code snippets and embedding nature language search queries.

Embeddings make it easier to do machine learning on large inputs representing words by capturing the semantic similarities in a vector space. Therefore, we can use embeddings to if two text chunks are semantically related or similar, and inherently provide a score to assess similarity.

### **Cosine Similarity**
A previously used approach to match similar documents was based on counting maximum number of common words between documents. This is flawed since as the document size increases, the overlap of common words increases even if the topics differ. Therefore cosine similarity is a better approach.

Mathematically, cosine similarity measures the cosine of the angle between two vectors projected in a multi-dimensional space. This is beneficial because if two documents are far apart by Euclidean distance because of size, they could still have a smaller angle between them and therefore higher cosine similarity.

The Azure OpenAI embeddings rely on cosine similarity to compute similarity between documents and a query.

## Setting up Azure OpenAI service and using deployed models

### **Chunking**

Let's start with chunking. Why is chunking important when working with LLMs?

Chunking helps overcome the challenges associated with processing long sequences and ensures optimal performance when working with LLMs.

**Mitigating Token Limitations:** LLMs have a maximum token limit for each input sequence. If a document or input exceeds this limit, it needs to be divided into chunks that fit within the token constraints. Chunking allows the LLM to handle long documents or inputs by splitting them into multiple chunks that fall within the token limit. This ensures that the model can effectively process the entire content while adhering to the token constraints.

**Memory and Computational Efficiency:** LLMs are computationally expensive and require substantial memory resources to process long sequences of text. Chunking involves breaking down long documents or input into smaller, manageable chunks, allowing the LLM to process them efficiently within its memory limitations. By dividing the input into smaller parts, chunking helps avoid memory errors or performance degradation that may occur when processing lengthy sequences.

**Contextual Coherence:** Chunking helps maintain contextual coherence in the generated outputs. Instead of treating the entire input as a single sequence, breaking it into smaller chunks allows the model to capture local context more effectively. This improves the model's understanding of the relationships and dependencies within the text, leading to more coherent and meaningful generated responses.

**Improved Parallelism:** Chunking enables parallel processing, which is essential for optimizing the performance of LLMs. By dividing the input into chunks, multiple chunks can be processed simultaneously, taking advantage of parallel computing capabilities. This leads to faster inference times and enhances overall efficiency when working with LLMs.

We will be leveraging a basic splitter for this notebook. However, it's important to note that there are more advanced splitters available, which may better suit your specific use case. 

In [ ]:
#Defining helper functions
#Splits text after sentences ending in a period. Combines n sentences per chunk.
def splitter(n, s):
    pieces = s.split(". ")
    list_out = [" ".join(pieces[i:i+n]) for i in range(0, len(pieces), n)]
    return list_out

# Perform light data cleaning (removing redudant whitespace and cleaning up punctuation)
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()    
    return s

In [ ]:
document_chunks = splitter(10, normalize_text(document)) #splitting extracted document into chunks of 10 sentences
document_chunks

In [ ]:
# Handling Rate Limits

from openai.error import RateLimitError
from time import sleep


def get_embedding(text: str, engine: str = "text-embedding-ada-002"):
    count=0
    while True:
        try:
            embedding = openai.Embedding().create(input=[text], engine=engine)["data"][0]["embedding"]
            break;
        except RateLimitError:
            count+=1
            #print(f'RateLimitError Count: {count}')
            sleep(2)            
    return np.array(embedding).astype(np.float32)

def get_completion(prompt, model="gpt-35-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


In [ ]:
embed_df = pd.DataFrame(document_chunks, columns = ["chunks"]) #datframe with document chunks

#Create an embedding vector for each chunk that will capture the semantic meaning and overall topic of that chunk
embed_df['embeddings'] = embed_df["chunks"].apply(lambda x : get_embedding(x, engine = embedding_model))

embed_df 


In [ ]:
# search through the document for a text segment most similar to the query
# display top two most similar chunks based on cosine similarity
def search_docs(df, user_query, top_n=3):
    embedding = get_embedding(
        user_query,
        engine=embedding_model,
    )
    df["similarities"] = df['embeddings'].apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .reset_index(drop=True)
        .head(top_n)
    )
    return res

In [ ]:
document_specific_query = "trouble so far in clinton campaign" 
res = search_docs(embed_df, document_specific_query, top_n=2) #finding top 2 results based on similarity 
res

## Section 3: Text Summarization

This section will cover the end-to-end flow of using the GPT-3 and ChatGPT models for summarization tasks. 
The model used by the Azure OpenAI service is a generative completion call which uses natural language instructions to identify the task being asked and skill required – aka Prompt Engineering. Using this approach, the first part of the prompt includes natural language instructions and/or examples of the specific task desired. The model then completes the task by predicting the most probable next text. This technique is known as "in-context" learning. 

There are three main approaches for in-context learning: Zero-shot, Few-shot and Fine tuning. These approaches vary based on the amount of task-specific data that is given to the model: 

**Zero-shot**: In this case, no examples are provided to the model and only the task request is provided. 

**Few-shot**: In this case, a user includes several examples in the call prompt that demonstrate the expected answer format and content. 

**Fine-Tuning**: Fine Tuning lets you tailor models to your personal datasets. This customization step will let you get more out of the service by providing: 
-	With lots of data (at least 500 and above) traditional optimization techniques are used with Back Propagation to re-adjust the weights of the model – this enables higher quality results than mere zero-shot or few-shot. 
-	A customized model improves the few-shot learning approach by training the model weights on your specific prompts and structure. This lets you achieve better results on a wider number of tasks without needing to provide examples in the prompt. The result is less text sent and fewer tokens 


In [ ]:
'''Designing a prompt that will show and tell GPT-3 how to proceed. 
+ Providing an instruction to summarize the text about the general topic (prefix)
+ Providing quality data for the chunks to summarize and specifically mentioning they are the text provided (context + context primer)
+ Providing a space for GPT-3 to fill in the summary to follow the format (suffix)
'''

# result_1 corresponding to the top chunk from Section 2. result_2 corresponding to the second to top chunk from section 2. 
# change index for desired chunk
result_1 = res.chunks[0]
result_2 = res.chunks[1]
prompt_i = 'Summarize the content about the Clinton campaign given the text provided.\n\nText:\n'+" ".join([normalize_text(result_1)])+ '\n\nText:\n'+ " ".join([normalize_text(result_2)])+'\n\nSummary:\n'
print(prompt_i)

In [ ]:
get_completion(prompt_i, model=chat_model) # default temperature is set to 0

In [ ]:
# bumping up the temperature to 0.5 to increase the randomness of the model's output
get_completion(prompt_i, model=chat_model, temperature=0.5)